In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action = 'ignore')
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.sans-serif']=['SimHei']  #解决中文显示乱码问题
plt.rcParams['axes.unicode_minus']=False
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn import tree
import sklearn.linear_model as LM
from sklearn import ensemble
from sklearn.datasets import make_classification,make_circles,make_regression
from sklearn.metrics import zero_one_loss,r2_score,mean_squared_error,accuracy_score
import xgboost as xgb

In [2]:
data=pd.read_excel('../数据/3-1训练集.xlsx')
data
# data=data.replace(0,np.NaN)
# data=data.dropna()
X=data.iloc[:,:-1]
X

,表面风化,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),氧化钙(CaO),氧化镁(MgO),氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2,二氧化硫(SO2)
0,1,69.33,0.00,9.99,6.32,0.87,3.93,1.74,3.87,0.00,0.00,1.17,0.00,0.0,0.39
1,1,87.05,0.00,5.19,2.01,0.00,4.06,0.00,0.78,0.25,0.00,0.66,0.00,0.0,0.00
2,1,61.71,0.00,12.37,5.87,1.11,5.50,2.16,5.09,1.41,2.86,0.70,0.10,0.0,0.00
3,1,65.88,0.00,9.67,7.12,1.56,6.44,2.06,2.18,0.00,0.00,0.79,0.00,0.0,0.36
4,1,61.58,0.00,10.95,7.35,1.77,7.50,2.62,3.27,0.00,0.00,0.94,0.06,0.0,0.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,2,17.11,0.00,0.00,0.00,1.11,3.65,0.00,1.34,58.46,0.00,14.13,1.12,0.0,0.00
65,1,49.01,2.71,0.00,1.13,0.00,1.45,0.00,0.86,32.92,7.95,0.35,0.00,0.0,0.00
66,2,29.15,0.00,0.00,1.21,0.00,1.85,0.00,0.79,41.25,15.45,2.54,0.00,0.0,0.00
67,2,25.42,0.00,0.00,1.31,0.00,2.18,0.00,1.16,45.10,17.30,0.00,0.00,0.0,0.00


In [3]:

Y=data['类型']

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,train_size=0.70, random_state=123)

In [ ]:
B=300
dt_stump = tree.DecisionTreeClassifier(max_depth=3, min_samples_leaf=1)
TestErrAdaB=np.zeros((B,))
adaBoost = ensemble.AdaBoostClassifier(base_estimator=dt_stump,n_estimators=B,random_state=123)
adaBoost.fit(X_train,Y_train)
for b,Y_pred in enumerate(adaBoost.staged_predict(X_test)):
    TestErrAdaB[b]=zero_one_loss(Y_test,Y_pred) 

TestErrGBRT=np.zeros((B,))
GBRT=ensemble.GradientBoostingClassifier(loss='deviance',n_estimators=B,max_depth=3,min_samples_leaf=1,random_state=123)
GBRT.fit(X_train,Y_train)
for b,Y_pred in enumerate(GBRT.staged_predict(X_test)):
    TestErrGBRT[b]=zero_one_loss(Y_test,Y_pred) 

TestErrBag=np.zeros((B,)) 
TestErrRF=np.zeros((B,))
for b in np.arange(B):
    Bag=ensemble.BaggingClassifier(base_estimator=dt_stump,n_estimators=b+1,oob_score=True,random_state=123,bootstrap=True)
    Bag.fit(X_train,Y_train)
    TestErrBag[b]=1-Bag.score(X_test,Y_test)
    RF=ensemble.RandomForestClassifier(max_depth=3,n_estimators=b+1,oob_score=True,random_state=123,
                                       bootstrap=True,max_features="auto")
    RF.fit(X_train,Y_train)
    TestErrRF[b]=1-RF.score(X_test,Y_test)


In [ ]:
plt.figure(figsize=(6,4))
plt.plot(np.arange(B),TestErrAdaB,linestyle='--',label="AdaBoost分类树",linewidth=1)
plt.plot(np.arange(B),TestErrGBRT,linestyle='-',label="梯度提升分类树",linewidth=2)
plt.plot(np.arange(B),TestErrBag,linestyle='-.',label="Bagging分类树",linewidth=2)
plt.plot(np.arange(B),TestErrRF,linestyle='-.',label="随机森林分类",linewidth=1)
plt.title("验证集",fontsize=12)
plt.xlabel("迭代次数",fontsize=12)
plt.ylabel("测试误差",fontsize=12)
plt.legend()   

In [ ]:
Xtrain=np.array(X_train)
Ytrain=np.array(Y_train)
Xtest=np.array(X_test)
Ytest=np.array(Y_test)
modelXGB=xgb.XGBClassifier(max_depth=3,learning_rate=1, n_estimators=B,objective='binary:logistic',random_state=123)
modelXGB.fit(Xtrain,Ytrain,eval_set=[(Xtrain, Ytrain), (Xtest, Ytest)],verbose=True)
result=modelXGB.evals_result()
print(accuracy_score(Ytrain,modelXGB.predict(Xtrain)))
print(1-zero_one_loss(Ytest,modelXGB.predict(Xtest)))   

In [ ]:
# result['validation_1']['logloss']

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(np.arange(B),TestErrAdaB,linestyle='--',label="AdaBoost分类树",linewidth=1)
plt.plot(np.arange(B),TestErrGBRT,linestyle='-',label="梯度提升分类树",linewidth=2)
plt.plot(np.arange(B),TestErrBag,linestyle='-.',label="Bagging分类树",linewidth=2)
plt.plot(np.arange(B),TestErrRF,linestyle='-.',label="随机森林分类",linewidth=1)
plt.plot(np.arange(B),result['validation_1']['logloss'],linestyle='-',label="XGBoost分类",linewidth=2)
plt.title("测试集",fontsize=15)
plt.xlabel("迭代次数",fontsize=15)
plt.ylabel("测试误差",fontsize=15)
plt.legend()   

In [ ]:
pr=pd.read_excel("../数据/3-1测试集.xlsx")
x=pr.iloc[:,1:]
RF.predict(x)

In [ ]:
GBRT.predict(x)

In [ ]:
adaBoost.predict(x)

In [ ]:
Bag.predict(x)